In [ ]:
#如果目录下不存在market.db文件，从上级目录复制一个过来
import shutil
import os
if not os.path.exists('market.db'):
    shutil.copyfile('../market.db', 'market.db')

In [ ]:
import sqlite3
import pandas as pd

from datetime import datetime
import pytz
def query(q_string, fname='market.db'):
    """
    Query the sqlite database and return a pandas dataframe
    """
    conn = sqlite3.connect(fname)
    c = conn.cursor()
    data = pd.read_sql_query(q_string, conn)
    conn.close()
    return data

def timestamp_to_beijing_time(ts):
    # 设置时区为北京（东八区）
    beijing_tz = pytz.timezone('Asia/Shanghai')

    # 将时间戳转为datetime对象并设置时区
    dt = datetime.fromtimestamp(ts, tz=pytz.utc).astimezone(beijing_tz)

    # 格式化输出：年月日 时:分:秒
    return dt.strftime('%Y-%m-%d %H:%M:%S')

# 示例
timestamp = 1745216102
print(timestamp_to_beijing_time(timestamp))
timestamp = 1745212502
print(timestamp_to_beijing_time(timestamp))

sql = 'select time from ask order by time desc limit 1'
ask_data = query(sql)
# 将时间戳转换为北京时间
timestamp = ask_data['time'].iloc[0]
beijing_time = timestamp_to_beijing_time(timestamp)
print(f"北京时间: {beijing_time}")


In [ ]:
import numpy as np
ask_data = query('SELECT * FROM ask')
ask_data.replace([np.inf, -np.inf, -1], np.nan, inplace=True)
# 按列进行插值处理（默认使用线性插值）
ask_data = ask_data.interpolate(method='linear', axis=0,limit_direction='both')

# 删除所有全为 NaN 的列
ask_data = ask_data.dropna(axis=1, how='all')

In [ ]:
# 计算每列的0.8分位数
quantile_80_values = ask_data.quantile(0.9)
# 对每一列进行处理，标记大于5倍中位数的值为NaN
ask_data = ask_data.apply(lambda x: np.where(x > 1.5 * quantile_80_values[x.name], np.nan, x))
# 转换为DataFrame以便查看
ask_data = pd.DataFrame(ask_data, columns=ask_data.columns)
# 按列进行插值处理（默认使用线性插值）
ask_data = ask_data.interpolate(method='linear', axis=0,limit_direction='both')


In [ ]:
# 计算每列的最大值
max_values = ask_data.max()
# 删除最大值大于100M的列(不包括第一列)
ask_data = ask_data.loc[:, (max_values < 1000000) | (ask_data.columns == 'time')]

In [ ]:
# 删除完全相同的列
ask_data = ask_data.loc[:, ~ask_data.T.duplicated()]


In [ ]:
# 计算每一行的总价格,每一行有765列，除了time列之外都是不同物品的价格
# 排除 'time' 列并计算每一行的总价格
total_price_list = ask_data.drop('time', axis=1).sum(axis=1)
ask_data['total_price_list'] = total_price_list
# 绘制总价格的走势图
import matplotlib.pyplot as plt
plt.plot(total_price_list)

In [ ]:
ask_data.shape

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

# 假设 df 是一个包含时间和商品价格的 DataFrame
# 第一列是时间列，剩下 764 列是商品价格

# 示例数据加载（需要根据实际数据修改）
df = ask_data.copy()  # 这里假设你的数据在 'data.csv' 文件中

# 假设数据的结构如下：第一列是日期，剩下的 764 列是商品价格
# df.shape 可能是 (1000, 765)，表示 1000 天的数据和 764 个商品价格

# 设定预测的时间步长和回溯的时间步长
n_past = 24*3
n_future = 2

# 准备输入和目标数据
def create_sequences(data, n_past, n_future):
    x, y = [], []
    for i in range(len(data) - n_past - n_future + 1):
        x.append(data[i:i+n_past])  # 获取过去 n_past 天的数据作为输入
        y.append(data[i+n_past:i+n_past+n_future])  # 获取接下来 n_future 天的数据作为目标
    return np.array(x), np.array(y)

# 假设 df 里每列（除了时间）是一个商品的价格
data = df.iloc[:, 1:].values# 排除时间列
# 把最后N条数据不进行训练
data = data[:-24]


In [ ]:
# 检查数据中是否有 NaN、Inf 或 -1 异常值
if np.any(np.isnan(data)) or np.any(np.isinf(data)) or np.any(data == -1):
    print("Data contains NaN, Inf, or -1!")
    print("has NaN:", np.any(np.isnan(data)))
    print("has Inf:", np.any(np.isinf(data)))
    print("has -1:", np.any(data == -1))

In [ ]:
# 使用 Z-score 标准化
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data = scaler.fit_transform(data)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
x_data, y_data = create_sequences(data, n_past, n_future)

# 将数据转化为 PyTorch Tensor
x_data = torch.tensor(x_data, dtype=torch.float32)
y_data = torch.tensor(y_data, dtype=torch.float32)

# 创建 PyTorch 数据集
class PriceDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

# 数据加载器
train_dataset = PriceDataset(x_data, y_data)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)


In [ ]:
import torch
import torch.nn as nn

class BiLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2, dropout=0.2):
        super(BiLSTMModel, self).__init__()
        # 双向 LSTM
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True, dropout=dropout)
        # BatchNorm 用于 LSTM 输出
        self.bn = nn.BatchNorm1d(2 * hidden_size)  # 因为是双向，hidden_size 需要乘以 2
        self.fc = nn.Linear(2 * hidden_size, output_size)  # 输出层大小要调整

    def forward(self, x):
        # LSTM 输出 (output, (h_n, c_n))
        out, (hn, cn) = self.lstm(x)
        # 双向 LSTM 会有 2 * hidden_size 输出
        out = out[:, -1, :]  # 取最后一个时间步
        out = self.bn(out)    # Batch Normalization
        out = self.fc(out)    # 全连接层
        return out

# 定义输入输出大小
input_size = x_data.shape[2]  # 商品价格的数量（764个商品）
hidden_size = 256  # LSTM 隐藏层的大小（增加隐藏层的大小）
output_size = n_future * input_size  # 预测未来 n_future 天的价格

# 实例化模型
model = BiLSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=3)

# 如果使用 GPU，记得将模型转到 GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
model = model.to(device)

In [ ]:
# 损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# 训练模型
num_epochs = 5000
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        # 将输入和目标转移到 GPU
        inputs = inputs.to(device)
        targets = targets.to(device)

        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, targets.view(targets.size(0), -1))  # 展开目标
        # 反向传播
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')


In [ ]:
# 测试模型
model.eval()
with torch.no_grad():
    # 假设我们使用训练数据中的最后一个时间窗口进行预测
    data = df.iloc[:, 1:].values# 排除时间列
    # 保留最后一个窗口
    data = data[-(n_past*1 + n_future*0):]
    data = scaler.transform(data)  # 使用相同的标准化器
    data = torch.tensor(data, dtype=torch.float32).unsqueeze(0)  # 添加批次维度
    data = data.to(device)
    predicted_prices = model(data)
    # 将输出的价格按商品拆分
    predicted_prices = predicted_prices.view(n_future, input_size).cpu().numpy()

In [ ]:
data.shape

In [ ]:
top_10_items_columns = df.columns[1:]  # 假设第1列以后的列是商品数据

price_changes = []
for i in range(input_size):
    # 计算每个商品的相对涨幅（从第一个时间点到最后一个）
    price_change = (predicted_prices[:, i] - predicted_prices[0, i]) / predicted_prices[0, i]
    price_changes.append(price_change)


# 假设 'price_changes' 存储的是每个商品的涨幅序列，并且已经提取了最终涨幅
final_price_changes = [float(change[-1]) for change in price_changes]

# 获取涨幅从大到小排序后的商品索引和涨幅
sorted_indices = sorted(range(len(final_price_changes)), key=lambda i: final_price_changes[i], reverse=True)

# 输出前 10 个可能涨价的商品
top_10_items = sorted_indices[:5]
for idx in top_10_items:
    print(f"{idx} 商品\t {top_10_items_columns[idx]} \t的涨幅为 {final_price_changes[idx]:.2%}")



In [ ]:
import pandas as pd

# 获取最后五天的数据

top_10_item_columns_to_display = top_10_items_columns[top_10_items]  # 根据前10个商品的索引筛选列

# 选择第一列（时间）和前10个商品的数据
columns_to_display = ['time'] + top_10_item_columns_to_display.tolist()  # '时间'列是第一列，假设第一列是时间

# 输出这几列的最后24h数据
last_5_days_data = df.loc[df.index[-24:], columns_to_display]
last_5_days_data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 假设 df 已经加载，并包含了时间列和商品价格数据
top_10_items_columns = df.columns[1:]  # 假设第1列以后的列是商品数据
top_10_item_columns_to_display = top_10_items_columns[top_10_items]  # 根据前10个商品的索引筛选列

# 选择第一列（时间）和前10个商品的数据
columns_to_display = ['time'] + top_10_item_columns_to_display.tolist()  # '时间'列是第一列，假设第一列是时间

# 获取最后24小时的数据
last_24_hours_data = df.loc[df.index[-24:], columns_to_display]  # 获取最后24行

# 归一化处理：每列的价格除以第一行的价格（确保每条曲线都从1开始）
for item in top_10_item_columns_to_display:
    last_24_hours_data[item] = last_24_hours_data[item] / last_24_hours_data[item].iloc[0]

# 绘制折线图
plt.figure(figsize=(12, 6))

# 对每个商品列绘制一条折线
for item in top_10_item_columns_to_display:
    plt.plot(last_24_hours_data['time'], last_24_hours_data[item], label=item)

# 设置图形标题和标签
plt.title('Top 10 Item Price Trends in the Last 24 Hours (Normalized)')
plt.xlabel('Time')
plt.ylabel('Normalized Price')
plt.legend(title="Items", bbox_to_anchor=(1.05, 1), loc='upper left')

# 显示图形
plt.tight_layout()
plt.show()


In [ ]:
last_5_days_data